<a href="https://colab.research.google.com/github/onedayxzn/Recommendation-movie/blob/master/SistemPenentuanKeputusanMenontonFilm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistem Penentuan Keputusan Menonton Film Dengan Algoritma Cosine_similarity 

Kelompok 5 
Anggota : 
*   Sukma Ramadhan Asri - 19552011045
*   Irvan Maulana       - 18111167
*   Orlando Sagala      - 17111109
*   Rifki Adriansyah    - 18111139
*   Reina Rosyi         - 18111286





### Project Overview  


---



Sistem Penentuan Keputusan Menonton Film / Movie merupakan sistem yang merekomendasikan movie kepada penonton, rekomendasi ini contohnya diterapkan pada 
situs seperti netflix, iqiyi, dan wetv. Sistem rekomendasi yang kami buat ini didasarkan dengan kesamaan genre pada suatu movie tersebut. Sistem rekomendasi telah menjadi lazim dalam beberapa tahun terakhir karena mereka menangani masalah kelebihan informasi dengan menyarankan pengguna produk yang paling relevan dari sejumlah besar data. Untuk produk media, rekomendasi movie dengan content based filtering berupaya membantu pengguna mengakses movie pilihan mereka dengan menangkap konten yang persis sama atau movie yang sesuai dengan genrenya. Namun, karena data yang jarang, pemilihan konten menjadi lebih sulit dengan meningkatnya movie dan pengguna dengan cepat.


In [ ]:
!pip install surprise

surprise merupakan library khusus collaborative filtering, library ini adalah 
salah satu perkembangan dari scikit-learn yang berfokus pada pembangunan dan analisa sistem rekomendasi

## Data Understanding

melakukan mounting dari google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


mengakses dataset dengan menggunakan library pandas

In [ ]:
import pandas as pd
movies = pd.read_csv('/content/drive/MyDrive/AISTTB/UAS/Dataset-Kelompok5-Tubes/Dataset-Movie.csv', encoding= 'unicode_escape')
ratings = pd.read_csv('/content/drive/MyDrive/AISTTB/UAS/Dataset-Kelompok5-Tubes/Dataset-Rating.csv',  encoding= 'unicode_escape')

print('jumlah shape data movies : ', movies.shape)
print('jumlah shape data ratings : ', ratings.shape)


jumlah shape data movies :  (1202, 4)
jumlah shape data ratings :  (1202, 2)


In [ ]:
movies.head(10)

,movie_id,movie_title,genre_ids,overview
0,NaN,NaN,NaN,NaN
1,821.0,Judgment at Nuremberg,[Drama|History|War],"In 1947, four German judges who served on the ..."
2,NaN,NaN,NaN,NaN
3,4176.0,Murder on the Orient Express,[Drama|Thriller|Mystery],"In 1935, when his train is stopped by deep sno..."
4,NaN,NaN,NaN,NaN
5,22584.0,To Have and Have Not,[Adventure|Comedy|Romance|Thriller|War],A Martinique charter boat skipper gets mixed u...
6,NaN,NaN,NaN,NaN
7,12584.0,The Shootist,[Western|Action|Adventure|Drama],Afflicted with a terminal illness John Bernard...
8,NaN,NaN,NaN,NaN
9,289.0,Casablanca,[Drama|Romance],"In Casablanca, Morocco in December 1941, a cyn..."


In [ ]:
ratings.head(10)

,movie_id,ratings
0,NaN,NaN
1,821.0,8.1
2,NaN,NaN
3,4176.0,7.2
4,NaN,NaN
5,22584.0,7.5
6,NaN,NaN
7,12584.0,7.2
8,NaN,NaN
9,289.0,8.2


In [ ]:
movies.info()
print('')
print('=============')
print('')
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202 entries, 0 to 1201
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie_id     601 non-null    float64
 1   movie_title  601 non-null    object 
 2   genre_ids    601 non-null    object 
 3   overview     601 non-null    object 
dtypes: float64(1), object(3)
memory usage: 37.7+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202 entries, 0 to 1201
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   movie_id  601 non-null    float64
 1   ratings   601 non-null    float64
dtypes: float64(2)
memory usage: 18.9 KB


## Data Cleaning

### data movies


cek jumlah baris dari masing-masing kolom yang memiliki nilai null atau kosong

In [ ]:
movies.isnull().sum()

movie_id       601
movie_title    601
genre_ids      601
overview       601
dtype: int64

menghapus baris yang memiliki nilai kosong atau null

In [ ]:
movies = movies.dropna()

In [ ]:
movies.isnull().sum()


movie_id       0
movie_title    0
genre_ids      0
overview       0
dtype: int64

mengubah tipe data pada kolom movie_id dari float menjadi int

In [ ]:
movies["movie_id"] = movies['movie_id'].astype('int')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
movies.rename(columns={'genre_ids':'genre'},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 601 entries, 1 to 1201
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movie_id     601 non-null    int64 
 1   movie_title  601 non-null    object
 2   genre        601 non-null    object
 3   overview     601 non-null    object
dtypes: int64(1), object(3)
memory usage: 23.5+ KB


In [ ]:
movies.head()

,movie_id,movie_title,genre,overview
1,821,Judgment at Nuremberg,[Drama|History|War],"In 1947, four German judges who served on the ..."
3,4176,Murder on the Orient Express,[Drama|Thriller|Mystery],"In 1935, when his train is stopped by deep sno..."
5,22584,To Have and Have Not,[Adventure|Comedy|Romance|Thriller|War],A Martinique charter boat skipper gets mixed u...
7,12584,The Shootist,[Western|Action|Adventure|Drama],Afflicted with a terminal illness John Bernard...
9,289,Casablanca,[Drama|Romance],"In Casablanca, Morocco in December 1941, a cyn..."


### data ratings

In [ ]:
ratings.isnull().sum()

movie_id    601
ratings     601
dtype: int64

In [ ]:
ratings = ratings.dropna()

In [ ]:
ratings.isnull().sum()

movie_id    0
ratings     0
dtype: int64

In [ ]:
ratings["movie_id"] = ratings['movie_id'].astype('int')
ratings["ratings"] = ratings['ratings'].astype('int')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
ratings.rename(columns={'ratings':'rating'},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 601 entries, 1 to 1201
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   movie_id  601 non-null    int64
 1   rating    601 non-null    int64
dtypes: int64(2)
memory usage: 14.1 KB


In [ ]:
ratings.head()

,movie_id,rating
1,821,8
3,4176,7
5,22584,7
7,12584,7
9,289,8


In [ ]:
ratings.describe()

,movie_id,rating
count,601.000000,601.000000
mean,39616.407654,6.317804
std,84156.456810,1.097193
min,11.000000,0.000000
25%,966.000000,6.000000
50%,10802.000000,6.000000
75%,37601.000000,7.000000
max,634649.000000,8.000000


## Data Preprocessing

In [ ]:
import numpy as np

# Menggabungkan seluruh movie_id pada kategori movie_all
movies_all = np.concatenate((
    movies.movie_id.unique(),
    ratings.movie_id.unique()
))

# Mengurutkan data dan menghapus data yang sama
movies_all = np.sort(np.unique(movies_all))
 
print('Jumlah seluruh data movie berdasarkan movie_id: ', len(movies_all))

Jumlah seluruh data movie berdasarkan movie_id:  536


Menggabungkan file movies dan ratings ke dalam dataframe movies_info, serta menggabungkan dataframa ratings dengan movies_info berdasarkan nilai pada movie_id

In [ ]:
movies_info = pd.concat([movies, ratings])
movies = pd.merge(movies_info, ratings , on='movie_id', how='left')
movies

,movie_id,movie_title,genre,overview,rating_x,rating_y
0,821,Judgment at Nuremberg,[Drama|History|War],"In 1947, four German judges who served on the ...",NaN,8
1,4176,Murder on the Orient Express,[Drama|Thriller|Mystery],"In 1935, when his train is stopped by deep sno...",NaN,7
2,22584,To Have and Have Not,[Adventure|Comedy|Romance|Thriller|War],A Martinique charter boat skipper gets mixed u...,NaN,7
3,12584,The Shootist,[Western|Action|Adventure|Drama],Afflicted with a terminal illness John Bernard...,NaN,7
4,289,Casablanca,[Drama|Romance],"In Casablanca, Morocco in December 1941, a cyn...",NaN,8
...,...,...,...,...,...,...
1509,27336,NaN,NaN,NaN,4.0,4
1510,40258,NaN,NaN,NaN,6.0,6
1511,133535,NaN,NaN,NaN,5.0,5
1512,1448,NaN,NaN,NaN,5.0,5


In [ ]:
movies.isnull().sum()

movie_id         0
movie_title    757
genre          757
overview       757
rating_x       757
rating_y         0
dtype: int64

menggabungkan rating berdasarkan movie_id

In [ ]:
movies.groupby('movie_id').sum()

,rating_x,rating_y
movie_id,,
11,72.0,144
12,7.0,14
24,8.0,16
58,7.0,14
62,32.0,64
...,...,...
497698,7.0,14
503706,4.0,8
524434,7.0,14





mendefinisikan variabel all_movie_rate dengan variabel ratings

In [ ]:
all_movies_rate = ratings
all_movies_rate

,movie_id,rating
1,821,8
3,4176,7
5,22584,7
7,12584,7
9,289,8
...,...,...
1193,27336,4
1195,40258,6
1197,133535,5
1199,1448,5


Menggabungkan all movies_rate dengan dataframe movies berdasarkan movie_id

In [ ]:
all_movies = pd.merge(movies[['movie_id','movie_title','genre','overview']], all_movies_rate, on='movie_id', how="right")
all_movies

,movie_id,movie_title,genre,overview,rating
0,821,Judgment at Nuremberg,[Drama|History|War],"In 1947, four German judges who served on the ...",8
1,821,NaN,NaN,NaN,8
2,4176,Murder on the Orient Express,[Drama|Thriller|Mystery],"In 1935, when his train is stopped by deep sno...",7
3,4176,NaN,NaN,NaN,7
4,22584,To Have and Have Not,[Adventure|Comedy|Romance|Thriller|War],A Martinique charter boat skipper gets mixed u...,7
...,...,...,...,...,...
2289,133535,NaN,NaN,NaN,5
2290,1448,The Wisdom of Crocodiles,[Drama|Fantasy|Horror],A vampire in London is searching for the ideal...,5
2291,1448,NaN,NaN,NaN,5
2292,5486,Ecologia del delitto,[Horror|Thriller],An elderly heiress is killed by her husband wh...,6


## Data Preparation

In [ ]:
all_movies.isnull().sum()

movie_id          0
movie_title    1147
genre          1147
overview       1147
rating            0
dtype: int64

In [ ]:
all_movies = all_movies.dropna()
all_movies

,movie_id,movie_title,genre,overview,rating
0,821,Judgment at Nuremberg,[Drama|History|War],"In 1947, four German judges who served on the ...",8
2,4176,Murder on the Orient Express,[Drama|Thriller|Mystery],"In 1935, when his train is stopped by deep sno...",7
4,22584,To Have and Have Not,[Adventure|Comedy|Romance|Thriller|War],A Martinique charter boat skipper gets mixed u...,7
6,12584,The Shootist,[Western|Action|Adventure|Drama],Afflicted with a terminal illness John Bernard...,7
8,289,Casablanca,[Drama|Romance],"In Casablanca, Morocco in December 1941, a cyn...",8
...,...,...,...,...,...
2284,27336,Lighthouse,[Horror|Thriller],A prison ship on its way to a remote island pr...,4
2286,40258,Baby Blood,[Comedy|Horror],A cruel circus owner beats and abuses his preg...,6
2288,133535,Aswang,[Horror],"A young, unwed, pregnant girl is made an offer...",5
2290,1448,The Wisdom of Crocodiles,[Drama|Fantasy|Horror],A vampire in London is searching for the ideal...,5


Mengurutkan movie berdasarkan movie_id kemudian memasukkannya ke dalam variabel fix_movie

In [ ]:
fix_movies = all_movies.sort_values('movie_id', ascending=True)
fix_movies

,movie_id,movie_title,genre,overview,rating
217,11,Star Wars,[Adventure|Action|Science Fiction],Princess Leia is captured and held hostage by ...,8
256,11,Star Wars,[Adventure|Action|Science Fiction],Princess Leia is captured and held hostage by ...,8
212,11,Star Wars,[Adventure|Action|ScienceFiction],Princess Leia is captured and held hostage by ...,8
213,11,Star Wars,[Adventure|Action|ScienceFiction],Princess Leia is captured and held hostage by ...,8
214,11,Star Wars,[Adventure|Action|ScienceFiction],Princess Leia is captured and held hostage by ...,8
...,...,...,...,...,...
1590,497698,Black Widow,[Action|Adventure|Science Fiction],"Natasha Romanoff, also known as Black Widow, c...",7
742,503706,The Haunting of Sharon Tate,[Horror|Thriller],Pregnant with director Roman Polanski's child ...,4
1586,524434,Eternals,[Action|Adventure|Fantasy|Science Fiction],The Eternals are a team of ancient aliens who ...,7
1588,566525,Shang-Chi and the Legend of the Ten Rings,[Action|Adventure|Fantasy],Shang-Chi must confront the past he thought he...,7


In [ ]:
len(fix_movies.movie_id.unique())

536

Selanjutnya, gunakan data unik untuk dimasukkan ke dalam proses pemodelan. serta hapus data duplicate dengan fungsi drop_duplicates() berdasarkan movie_id

In [ ]:
fix_movies = fix_movies.drop_duplicates('movie_id')
fix_movies

,movie_id,movie_title,genre,overview,rating
217,11,Star Wars,[Adventure|Action|Science Fiction],Princess Leia is captured and held hostage by ...,8
2068,12,Finding Nemo,[Animation|Family],"Nemo, an adventurous young clownfish, is unexp...",7
1910,24,Kill Bill: Vol. 1,[Action|Crime],"An assassin is shot by her ruthless employer, ...",8
2048,58,Pirates of the Caribbean: Dead Man's Chest,[Adventure|Fantasy|Action],Captain Jack Sparrow works his way out of a bl...,7
1433,62,2001: A Space Odyssey,[Science Fiction|Mystery|Adventure],Humanity finds a mysterious object buried bene...,8
...,...,...,...,...,...
1590,497698,Black Widow,[Action|Adventure|Science Fiction],"Natasha Romanoff, also known as Black Widow, c...",7
742,503706,The Haunting of Sharon Tate,[Horror|Thriller],Pregnant with director Roman Polanski's child ...,4
1586,524434,Eternals,[Action|Adventure|Fantasy|Science Fiction],The Eternals are a team of ancient aliens who ...,7
1588,566525,Shang-Chi and the Legend of the Ten Rings,[Action|Adventure|Fantasy],Shang-Chi must confront the past he thought he...,7


In [ ]:
fix_movies.isnull().sum()

movie_id       0
movie_title    0
genre          0
overview       0
rating         0
dtype: int64

Selanjutnya, melakukan konversi data series menjadi list. Dalam hal ini, menggunakan fungsi tolist() dari library numpy

In [ ]:
movieId = fix_movies['movie_id'].tolist()
movieTitle = fix_movies['movie_title'].tolist()
movieGenre = fix_movies['genre'].tolist()
movieOverview = fix_movies['overview'].tolist()
movieRatings = fix_movies['rating'].tolist()

print('banyak movieId', len(movieId))
print('banyak movieTitle', len(movieTitle))
print('banyak movieGenre', len(movieGenre))
print('banyak movieOverview', len(movieOverview))
print('banyak movieRatings', len(movieRatings))

banyak movieId 536
banyak movieTitle 536
banyak movieGenre 536
banyak movieOverview 536
banyak movieRatings 536


membuat dictionary untuk menentukan pasangan key-value pada data movieId, movieTitle, dan movieGenre yang telah disiapkan sebelumnya.

In [ ]:
movies_new = pd.DataFrame({
    'id' : movieId,
    'title' : movieTitle,
    'genre' : movieGenre,
    'overview' : movieOverview,
    'rating' : movieRatings
})

movies_new

,id,title,genre,overview,rating
0,11,Star Wars,[Adventure|Action|Science Fiction],Princess Leia is captured and held hostage by ...,8
1,12,Finding Nemo,[Animation|Family],"Nemo, an adventurous young clownfish, is unexp...",7
2,24,Kill Bill: Vol. 1,[Action|Crime],"An assassin is shot by her ruthless employer, ...",8
3,58,Pirates of the Caribbean: Dead Man's Chest,[Adventure|Fantasy|Action],Captain Jack Sparrow works his way out of a bl...,7
4,62,2001: A Space Odyssey,[Science Fiction|Mystery|Adventure],Humanity finds a mysterious object buried bene...,8
...,...,...,...,...,...
531,497698,Black Widow,[Action|Adventure|Science Fiction],"Natasha Romanoff, also known as Black Widow, c...",7
532,503706,The Haunting of Sharon Tate,[Horror|Thriller],Pregnant with director Roman Polanski's child ...,4
533,524434,Eternals,[Action|Adventure|Fantasy|Science Fiction],The Eternals are a team of ancient aliens who ...,7
534,566525,Shang-Chi and the Legend of the Ten Rings,[Action|Adventure|Fantasy],Shang-Chi must confront the past he thought he...,7


## Modeling and Result

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
tf.fit(movies_new['genre'])
tf.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['action',
 'adventure',
 'animation',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'family',
 'fantasy',
 'fiction',
 'history',
 'horror',
 'music',
 'mystery',
 'romance',
 'science',
 'sciencefiction',
 'thriller',
 'war',
 'western']

melakukan fit dan transformasi kedalam bentuk matriks

In [ ]:
tfidf_matrix = tf.fit_transform(movies_new['genre'])
tfidf_matrix.shape

(536, 20)

menghasilkan vektor tf-idf dalam bentuk matriks, menggunakan fungsi todense().

In [ ]:
tfidf_matrix.todense()

matrix([[0.45884614, 0.47416279, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.75886116, ..., 0.        , 0.        ,
         0.        ],
        [0.57331526, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.39434512, 0.40750868, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.51648637, 0.5337271 , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.45884614, 0.47416279, 0.        , ..., 0.        , 0.        ,
         0.        ]])

lihat matriks tf-idf untuk beberapa movie (movie_title) dan genre

In [ ]:
pd.DataFrame(
    tfidf_matrix.todense(),
    columns=tf.get_feature_names(),
    index=movies_new.title
).sample(20, axis=1).sample(10, axis=0)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,history,family,horror,mystery,action,science,crime,romance,western,animation,war,documentary,fiction,drama,music,sciencefiction,adventure,thriller,fantasy,comedy
title,,,,,,,,,,,,,,,,,,,,
King Kong,0.0,0.0,0.561619,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.515714,0.000000,0.647011,0.0
The Poseidon Adventure,0.0,0.0,0.000000,0.0,0.490198,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.481161,0.0,0.0,0.506561,0.521139,0.000000,0.0
The China Syndrome,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.678363,0.0,0.0,0.000000,0.734727,0.000000,0.0
Funny Games,0.0,0.0,0.613924,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.535476,0.0,0.0,0.000000,0.579968,0.000000,0.0
...E tu vivrai nel terrore! L'aldilà,0.0,0.0,1.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0
Psycho,0.0,0.0,0.613924,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.535476,0.0,0.0,0.000000,0.579968,0.000000,0.0
The Dentist,0.0,0.0,0.726924,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.686718,0.000000,0.0
American Reunion,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1.0
Henry: Portrait of a Serial Killer,0.0,0.0,0.484167,0.0,0.000000,0.0,0.614851,0.0,0.000000,0.0,0.0,0.0,0.0,0.422300,0.0,0.0,0.000000,0.457388,0.000000,0.0


Selanjutnya, kita akan menghitung derajat kesamaan antara satu judul movie dengan movie lainnya untuk menghasilkan kandidat movie mana yang akan direkomendasikan.

### Cosine Similiarity
yaitu sebuah algoritma untuk menghitung derajat kesamaan (similiarity degree)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix) 
cosine_sim

array([[1.        , 0.        , 0.26306349, ..., 0.8594278 , 0.4900613 ,
        1.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.26306349, 0.        , 1.        , ..., 0.22608408, 0.29610951,
        0.26306349],
       ...,
       [0.8594278 , 0.        , 0.22608408, ..., 1.        , 0.76351507,
        0.8594278 ],
       [0.4900613 , 0.        , 0.29610951, ..., 0.76351507, 1.        ,
        0.4900613 ],
       [1.        , 0.        , 0.26306349, ..., 0.8594278 , 0.4900613 ,
        1.        ]])

In [ ]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=movies_new['title'], columns=movies_new['title'])
print('shape: ', cosine_sim_df.shape)

cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

shape:  (536, 536)


title,Die Hard,Justice League,Night Patrol,El Dorado,To Have and Have Not
title,,,,,
A Serious Man,0.000000,0.000000,0.785088,0.000000,0.310959
Spice World,0.000000,0.312840,0.359081,0.107703,0.240343
The Dirty Dozen,0.277691,0.330503,0.000000,0.292880,0.656450
Avatar,0.270187,1.000000,0.000000,0.284965,0.134062
Let's Scare Jessica to Death,0.000000,0.000000,0.000000,0.000000,0.000000
The Exorcist,0.000000,0.000000,0.000000,0.000000,0.000000
Milk,0.000000,0.000000,0.000000,0.000000,0.000000
Dogma,0.000000,0.522927,0.600221,0.180031,0.401745
X-Men: Apocalypse,0.270187,1.000000,0.000000,0.284965,0.134062




Dengan cosine similarity, kita berhasil mengidentifikasi kesamaan antara satu judul movie dengan judul movie lainnya. Shape (536, 536) merupakan ukuran matriks similarity dari data yang di miliki. Berdasarkan data yang ada, matriks di atas sebenarnya berukuran 536 judul movie x 536 judul movie (masing-masing dalam sumbu X dan Y). Artinya, mengidentifikasi tingkat kesamaan pada 536 judul movie. Tapi tentu tidak bisa menampilkan semuanya. Oleh karena itu, saya hanya memilih 10 movie pada baris vertikal dan 5 movie pada sumbu horizontal seperti pada contoh di atas.


Selanjutnya adalah mencoba mendapatkan rekomendasi .

Di sini, kita akan membuat fungsi film_rekomendasi dengan beberapa parameter sebagai berikut:

- title : judul movie (index kemiripan dataframe).

- Similarity_data : Dataframe mengenai similarity yang telah saya definisikan sebelumnya.

- Items : Nama dan fitur yang digunakan untuk mendefinisikan kemiripan, dalam hal ini adalah ‘title’ dan ‘genre’.

- k : Banyak rekomendasi yang ingin diberikan.

Definisi sistem rekomendasi yang menyatakan bahwa keluaran sistem ini adalah berupa top-N recommendation. Oleh karena itu, kita akan memberikan sejumlah rekomendasi judul movie pada pengguna yang diatur dalam parameter k.

In [ ]:
def movie_rekomendasi(nama_movie, similarity_data=cosine_sim_df, items=movies_new[['title', 'genre', 'rating']], k=5):
   index = similarity_data.loc[:,nama_movie].to_numpy().argpartition(
        range(-1, -k, -1))
   closest = similarity_data.columns[index[-1:-(k+2):-1]]
   closest = closest.drop(nama_movie, errors='ignore')
   return pd.DataFrame(closest).merge(items).head(k)

In [ ]:
judul_movie = input('masukan judul movie yang ingin di cek : ')

masukan judul movie yang ingin di cek : Pin


In [ ]:
movies_new[movies_new.title.eq(judul_movie)]

,id,title,genre,overview,rating
421,41966,Pin,[Horror|Thriller],"A doctor has a lifelike, anatomically-correct ...",6


In [ ]:
movie_rekomendasi(judul_movie)

,title,genre,rating
0,I Know Who Killed Me,[Thriller|Horror],4
1,Night of the Living Dead,[Horror|Thriller],7
2,The Dentist,[Horror|Thriller],5
3,The Last House on the Left,[Horror|Thriller],5
4,The Ugly,[Horror|Thriller],5


In [ ]:
from surprise import NormalPredictor
from surprise.model_selection import KFold
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

data = Dataset.load_from_df(movies_new[['title', 'genre', 'rating']], Reader())
kf = KFold(n_splits=5)
kf.split(data)

<generator object KFold.split at 0x7f147bb20ed0>

In [ ]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.6942  1.7321  1.6611  1.6499  1.5575  1.6044  1.5480  1.7483  1.7101  1.7910  1.6697  0.0769  
MAE (testset)     1.3519  1.4074  1.4630  1.4259  1.2778  1.3519  1.3396  1.5094  1.4528  1.5849  1.4165  0.0861  
Fit time          0.03    0.03    0.03    0.03    0.03    0.03    0.03    0.03    0.03    0.03    0.03    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'fit_time': (0.03288102149963379,
  0.030270814895629883,
  0.029130220413208008,
  0.028572559356689453,
  0.028769969940185547,
  0.02873826026916504,
  0.031829118728637695,
  0.03155374526977539,
  0.02912735939025879,
  0.03052520751953125),
 'test_mae': array([1.35185185, 1.40740741, 1.46296296, 1.42592593, 1.27777778,
        1.35185185, 1.33962264, 1.50943396, 1.45283019, 1.58490566]),
 'test_rmse': array([1.69421674, 1.73205081, 1.66110182, 1.64991582, 1.55753842,
        1.60439212, 1.54797494, 1.74831455, 1.71012523, 1.79096264]),
 'test_time': (0.00041747093200683594,
  0.000438690185546875,
  0.0004329681396484375,
  0.00040221214294433594,
  0.0003924369812011719,
  0.0004374980926513672,
  0.00040078163146972656,
  0.00039649009704589844,
  0.00044655799865722656,
  0.0004177093505859375)}